In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]


# 1. Time Series All

In [22]:
import panel as pn

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)

def get_line_plot():
    return data.hvplot.line(width=600, height=600, grid=True)

time_title = '## Time-Series'

time_row = pn.Row(
    pn.Column(time_title),
    get_line_plot()
)

time_row.servable()

Row
    [0] Column
        [0] Markdown(str)
    [1] HoloViews(NdOverlay, height=600, sizing_mode='fixed', width=600)

# 2. Time Series Volume Selector

In [19]:


def get_line_plot(volume):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    return df.hvplot.line(width=600, grid=True)

time_title = '## Time-Series'
volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)

time_row = pn.Row(
    pn.Column(time_title, volume),
    get_line_plot(volume.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value)

volume.param.watch(update_time_row, 'value')

time_row.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Volume', options=['high', 'medium', ...], value='high', width=200)
    [1] HoloViews(NdOverlay, height=300, sizing_mode='fixed', width=600)

# 3. Time Series Volume Selector, Average Slider

In [21]:
def get_line_plot(volume, window_size):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.rolling(window=window_size).mean() # calculate rolling average
    return df.hvplot.line(frame_width=600, grid=True)

time_title = '## Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg),
    get_line_plot(volume.value, day_avg.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')

time_row.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Volume', options=['high', 'medium', ...], value='high', width=200)
        [2] IntSlider(end=21, name='Day average', start=1, value=7, width=200)
    [1] HoloViews(NdOverlay, height=300, sizing_mode='fixed')

# 4. Time Series Selector, Average Slider, Date Slider

In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


# %%
def get_line_plot(volume, window_size, date_range):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.rolling(window=window_size).mean() # calculate rolling average
    df = df.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.line(frame_width=600, grid=True)

# %%
time_title = '## Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)
time_date_range = pn.widgets.DateRangeSlider(name='Dates', width=200,
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1])

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg, time_date_range),
    get_line_plot(volume.value, day_avg.value, time_date_range.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value, time_date_range.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')
time_date_range.param.watch(update_time_row, 'value')

time_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]


Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Volume', options=['high', 'medium', ...], value='high', width=200)
        [2] IntSlider(end=21, name='Day average', start=1, value=7, width=200)
        [3] DateRangeSlider(end=Timestamp('2019-12-31 0..., name='Dates', start=Timestamp('2019-01-01 0..., value=(Timestamp('2019-01-01 00:..., value_end=Timestamp('2019-12-31 0..., value_start=Timestamp('2019-01-01 0..., width=200)
    [1] HoloViews(NdOverlay, height=300, sizing_mode='fixed')

# 5. Correlation

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


# %%
def get_correlation_plots(product_x, product_y):
    df = data
    return df.hvplot.scatter(frame_height=300, frame_width=300,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1)

# %%
correlation_title = '## Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y),
    get_correlation_plots(product_x.value, product_y.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')

correlation_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]


Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='x-axis', options=['A', 'B', 'C', ...], value='A', width=200)
        [2] Select(name='y-axis', options=['A', 'B', 'C', ...], value='A', width=200)
    [1] HoloViews(Scatter, sizing_mode='fixed')

# 6. Correlation Heat Map

In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


# %%
def get_correlation_plots(product_x, product_y):
    df = data
    return df.hvplot.scatter(frame_height=300, frame_width=300,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=300, frame_width=300,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))

# %%
correlation_title = '## Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y),
    get_correlation_plots(product_x.value, product_y.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')

correlation_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]


Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='x-axis', options=['A', 'B', 'C', ...], value='A', width=200)
        [2] Select(name='y-axis', options=['A', 'B', 'C', ...], value='A', width=200)
    [1] HoloViews(Layout)

# 7. Correlation Heat Map, Date Slider

In [26]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


# %%
def get_correlation_plots(product_x, product_y, date_range):
    df = data.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.scatter(frame_height=300, frame_width=300,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=300, frame_width=300,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))

# %%
correlation_title = '## Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)
corr_date_range = pn.widgets.DateRangeSlider(name='Dates',
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1],
                                        width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y, corr_date_range),
    get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')
corr_date_range.param.watch(update_correlation_row, 'value')

correlation_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]


Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='x-axis', options=['A', 'B', 'C', ...], value='A', width=200)
        [2] Select(name='y-axis', options=['A', 'B', 'C', ...], value='A', width=200)
        [3] DateRangeSlider(end=Timestamp('2019-12-31 0..., name='Dates', start=Timestamp('2019-01-01 0..., value=(Timestamp('2019-01-01 00:..., value_end=Timestamp('2019-12-31 0..., value_start=Timestamp('2019-01-01 0..., width=200)
    [1] HoloViews(Layout)

# 8. Variables

In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn
import math

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

marketing_data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/MarketingPerProduct.csv', index_col=0)
price_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/PricePerUnit.csv', index_col=0)
profit_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/ProfitPerUnit.csv', index_col=0)

summary_data = pd.DataFrame(index=data.columns)
summary_data['Price'] = price_per_unit.values
summary_data['Profit'] = profit_per_unit.values
summary_data['Sales'] = data.sum().values
summary_data['Marketing'] = marketing_data.values
summary_data['Advertising'] = summary_data['Marketing'] / summary_data['Sales']
summary_data['Cost'] = summary_data['Price'] - summary_data['Profit']
print(summary_data.head())
print(summary_data.describe())

variables = list(summary_data.columns)

# %%
def get_summary_plots(variable_x, variable_y, variable_z):
    summary_data['BubbleSize'] = summary_data[variable_z] / summary_data[variable_z].max() * 200
    return summary_data.hvplot.scatter(frame_height=300, frame_width=300,
                                       title=variable_x + ' vs ' + variable_y + ' (vs ' + variable_z + ')',
                                       alpha=0.5, hover_cols='all', padding=0.1,
                                       x=variable_x, y=variable_y, size='BubbleSize')           

# %%
variables_title = '## Variables'

variable_x = pn.widgets.Select(name='X-axis', options=variables, value=variables[0], width=200)
variable_y = pn.widgets.Select(name='Y-axis', options=variables, value=variables[1], width=200)
variable_z = pn.widgets.Select(name='Bubble', options=variables, value=variables[2], width=200)

variables_row = pn.Row(
    pn.Column(variables_title, variable_x, variable_y, variable_z),
    get_summary_plots(variable_x.value, variable_y.value, variable_z.value)
)

def update_variables_row(event):
    variables_row[1].object = get_summary_plots(variable_x.value, variable_y.value, variable_z.value)

variable_x.param.watch(update_variables_row, 'value')
variable_y.param.watch(update_variables_row, 'value')
variable_z.param.watch(update_variables_row, 'value')

variables_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]
   Price  Profit   Sales  Marketing  Advertising  Cost
A     21       6  175173      21000     0.119881    15
B     28      16    4489       3000     0.668300    12

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='X-axis', options=['Price', 'Profit', ...], value='Price', width=200)
        [2] Select(name='Y-axis', options=['Price', 'Profit', ...], value='Profit', width=200)
        [3] Select(name='Bubble', options=['Price', 'Profit', ...], value='Sales', width=200)
    [1] HoloViews(Scatter, sizing_mode='fixed')

# 9. Variables Bubble Scaling

In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn
import math

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

marketing_data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/MarketingPerProduct.csv', index_col=0)
price_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/PricePerUnit.csv', index_col=0)
profit_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/ProfitPerUnit.csv', index_col=0)

summary_data = pd.DataFrame(index=data.columns)
summary_data['Price'] = price_per_unit.values
summary_data['Profit'] = profit_per_unit.values
summary_data['Sales'] = data.sum().values
summary_data['Marketing'] = marketing_data.values
summary_data['Advertising'] = summary_data['Marketing'] / summary_data['Sales']
summary_data['Cost'] = summary_data['Price'] - summary_data['Profit']
print(summary_data.head())
print(summary_data.describe())

variables = list(summary_data.columns)

# %%
def get_summary_plots(variable_x, variable_y, variable_z, bubble_scaling):
    summary_data['BubbleSize'] = summary_data[variable_z] / summary_data[variable_z].max() * math.exp(bubble_scaling)
    return summary_data.hvplot.scatter(frame_height=300, frame_width=300,
                                       title=variable_x + ' vs ' + variable_y + ' (vs ' + variable_z + ')',
                                       alpha=0.5, hover_cols='all', padding=0.1,
                                       x=variable_x, y=variable_y, size='BubbleSize')

# %%
variables_title = '## Variables'

variable_x = pn.widgets.Select(name='X-axis', options=variables, value=variables[0], width=200)
variable_y = pn.widgets.Select(name='Y-axis', options=variables, value=variables[1], width=200)
variable_z = pn.widgets.Select(name='Bubble', options=variables, value=variables[2], width=200)
bubble_scaling = pn.widgets.IntSlider(name='Bubble scaling', value=5, start=0, end=10, width=200)

variables_row = pn.Row(
    pn.Column(variables_title, variable_x, variable_y, variable_z, bubble_scaling),
    get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)
)

def update_variables_row(event):
    variables_row[1].object = get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)

variable_x.param.watch(update_variables_row, 'value')
variable_y.param.watch(update_variables_row, 'value')
variable_z.param.watch(update_variables_row, 'value')
bubble_scaling.param.watch(update_variables_row, 'value')

variables_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]
   Price  Profit   Sales  Marketing  Advertising  Cost
A     21       6  175173      21000     0.119881    15
B     28      16    4489       3000     0.668300    12

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='X-axis', options=['Price', 'Profit', ...], value='Price', width=200)
        [2] Select(name='Y-axis', options=['Price', 'Profit', ...], value='Profit', width=200)
        [3] Select(name='Bubble', options=['Price', 'Profit', ...], value='Sales', width=200)
        [4] IntSlider(end=10, name='Bubble scaling', value=5, width=200)
    [1] HoloViews(Scatter, sizing_mode='fixed')

# 10. Dashboard Standalone

In [29]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn
import math

pn.extension(comms='colab')

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

marketing_data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/MarketingPerProduct.csv', index_col=0)
price_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/PricePerUnit.csv', index_col=0)
profit_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/ProfitPerUnit.csv', index_col=0)

summary_data = pd.DataFrame(index=data.columns)
summary_data['Price'] = price_per_unit.values
summary_data['Profit'] = profit_per_unit.values
summary_data['Sales'] = data.sum().values
summary_data['Marketing'] = marketing_data.values
summary_data['Advertising'] = summary_data['Marketing'] / summary_data['Sales']
summary_data['Cost'] = summary_data['Price'] - summary_data['Profit']
print(summary_data.head())
print(summary_data.describe())

variables = list(summary_data.columns)

# %%
def get_line_plot(volume, window_size, date_range):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    df = df.rolling(window=window_size).mean() # calculate rolling average
    return df.hvplot.line(frame_height=200, frame_width=600, grid=True)


time_title = '### Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)
time_date_range = pn.widgets.DateRangeSlider(name='Dates', width=200,
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1])

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg, time_date_range),
    get_line_plot(volume.options[0], day_avg.value, time_date_range.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value, time_date_range.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')
time_date_range.param.watch(update_time_row, 'value')

#time_row.servable()

# %%
def get_correlation_plots(product_x, product_y, date_range):
    df = data.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.scatter(frame_height=250, frame_width=250,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=250, frame_width=250,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))


correlation_title = '### Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)
corr_date_range = pn.widgets.DateRangeSlider(name='Dates',
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1],
                                        width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y, corr_date_range),
    get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')
corr_date_range.param.watch(update_correlation_row, 'value')

#correlation_row.servable()

# %%
def get_summary_plots(variable_x, variable_y, variable_z, bubble_scaling):
    summary_data['BubbleSize'] = summary_data[variable_z] / summary_data[variable_z].max() * math.exp(bubble_scaling)
    return summary_data.hvplot.scatter(frame_height=250, frame_width=250,
                                       title=variable_x + ' vs ' + variable_y + ' (vs ' + variable_z + ')',
                                       alpha=0.5, hover_cols='all', padding=0.1,
                                       x=variable_x, y=variable_y, size='BubbleSize') 


variables_title = '### Variables'

variable_x = pn.widgets.Select(name='X-axis', options=variables, value=variables[0], width=200)
variable_y = pn.widgets.Select(name='Y-axis', options=variables, value=variables[1], width=200)
variable_z = pn.widgets.Select(name='Bubble', options=variables, value=variables[2], width=200)
bubble_scaling = pn.widgets.IntSlider(name='Bubble scaling', value=5, start=0, end=10, width=200)

variables_row = pn.Row(
    pn.Column(variables_title, variable_x, variable_y, variable_z, bubble_scaling),
    get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)
)

def update_variables_row(event):
    variables_row[1].object = get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)

variable_x.param.watch(update_variables_row, 'value')
variable_y.param.watch(update_variables_row, 'value')
variable_z.param.watch(update_variables_row, 'value')
bubble_scaling.param.watch(update_variables_row, 'value')

#variables_row.servable()

# %%
# the widgets in this cell will only work if you comment out the .servable() rows in the cells above
dashboard = pn.Column(time_row, correlation_row, variables_row)
dashboard

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]
   Price  Profit   Sales  Marketing  Advertising  Cost
A     21       6  175173      21000     0.119881    15
B     28      16    4489       3000     0.668300    12

Column
    [0] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(name='Volume', options=['high', 'medium', ...], value='high', width=200)
            [2] IntSlider(end=21, name='Day average', start=1, value=7, width=200)
            [3] DateRangeSlider(end=Timestamp('2019-12-31 0..., name='Dates', start=Timestamp('2019-01-01 0..., value=(Timestamp('2019-01-01 00:..., value_end=Timestamp('2019-12-31 0..., value_start=Timestamp('2019-01-01 0..., width=200)
        [1] HoloViews(NdOverlay, sizing_mode='fixed')
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(name='x-axis', options=['A', 'B', 'C', ...], value='A', width=200)
            [2] Select(name='y-axis', options=['A', 'B', 'C', ...], value='A', width=200)
            [3] DateRangeSlider(end=Timestamp('2019-12-31 0..., name='Dates', start=Timestamp('2019-01-01 0..., value=(Timestamp('2019-01-01 00:..., value_end=Timestamp('2019-12-31 0..., value_start=Timestamp('2019-01-01 0..., width=200)
        [1] HoloViews(Layout)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(name='X-axis', options=['Price', 'Profit', ...], value='Price', width=200)
            [2] Select(name='Y-axis', options=['Price', 'Profit', ...], value='Profit', width=200)
            [3] Select(name='Bubble', options=['Price', 'Profit', ...], value='Sales', width=200)
            [4] IntSlider(end=10, name='Bubble scaling', value=5, width=200)
        [1] HoloViews(Scatter, sizing_mode='fixed')

# 11. Dashboard Jupyter Bokeh

In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn
import math
import bokeh

pn.extension()

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())

marketing_data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/MarketingPerProduct.csv', index_col=0)
price_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/PricePerUnit.csv', index_col=0)
profit_per_unit = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/ProfitPerUnit.csv', index_col=0)

summary_data = pd.DataFrame(index=data.columns)
summary_data['Price'] = price_per_unit.values
summary_data['Profit'] = profit_per_unit.values
summary_data['Sales'] = data.sum().values
summary_data['Marketing'] = marketing_data.values
summary_data['Advertising'] = summary_data['Marketing'] / summary_data['Sales']
summary_data['Cost'] = summary_data['Price'] - summary_data['Profit']
print(summary_data.head())
print(summary_data.describe())

variables = list(summary_data.columns)

# %%
def get_line_plot(volume, window_size, date_range):
    if volume == 'high':
        selected = ['A', 'F', 'L']
    elif volume == 'medium':
        selected = ['G', 'H', 'J', 'S', 'W']
    elif volume == 'low':
        selected = ['D', 'E', 'M', 'O', 'P', 'T', 'X']
    else:
        selected = ['B', 'C', 'I', 'K', 'N', 'Q', 'R', 'U', 'V', 'Y']
    df = data[selected] # restrict to selected products
    df = df.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    df = df.rolling(window=window_size).mean() # calculate rolling average
    return df.hvplot.line(frame_height=200, frame_width=600, grid=True)


time_title = '### Time-Series'

volumes = ['high', 'medium', 'low', 'very low']

volume = pn.widgets.Select(name='Volume', options=volumes, width=200)
day_avg = pn.widgets.IntSlider(name='Day average', value=7, start=1, end=21, width=200)
time_date_range = pn.widgets.DateRangeSlider(name='Dates', width=200,
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1])

time_row = pn.Row(
    pn.Column(time_title, volume, day_avg, time_date_range),
    get_line_plot(volume.options[0], day_avg.value, time_date_range.value)
)

def update_time_row(event):
    time_row[1].object = get_line_plot(volume.value, day_avg.value, time_date_range.value)

volume.param.watch(update_time_row, 'value')
day_avg.param.watch(update_time_row, 'value')
time_date_range.param.watch(update_time_row, 'value')

time_row.servable()

# %%
def get_correlation_plots(product_x, product_y, date_range):
    df = data.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.scatter(frame_height=250, frame_width=250,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=250, frame_width=250,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))


correlation_title = '### Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)
corr_date_range = pn.widgets.DateRangeSlider(name='Dates',
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1],
                                        width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y, corr_date_range),
    get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')
corr_date_range.param.watch(update_correlation_row, 'value')

correlation_row.servable()

# %%
def get_summary_plots(variable_x, variable_y, variable_z, bubble_scaling):
    summary_data['BubbleSize'] = summary_data[variable_z] / summary_data[variable_z].max() * math.exp(bubble_scaling)
    return summary_data.hvplot.scatter(frame_height=250, frame_width=250,
                                       title=variable_x + ' vs ' + variable_y + ' (vs ' + variable_z + ')',
                                       alpha=0.5, hover_cols='all', padding=0.1,
                                       x=variable_x, y=variable_y, size='BubbleSize') 


variables_title = '### Variables'

variable_x = pn.widgets.Select(name='X-axis', options=variables, value=variables[0], width=200)
variable_y = pn.widgets.Select(name='Y-axis', options=variables, value=variables[1], width=200)
variable_z = pn.widgets.Select(name='Bubble', options=variables, value=variables[2], width=200)
bubble_scaling = pn.widgets.IntSlider(name='Bubble scaling', value=5, start=0, end=10, width=200)

variables_row = pn.Row(
    pn.Column(variables_title, variable_x, variable_y, variable_z, bubble_scaling),
    get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)
)

def update_variables_row(event):
    variables_row[1].object = get_summary_plots(variable_x.value, variable_y.value, variable_z.value, bubble_scaling.value)

variable_x.param.watch(update_variables_row, 'value')
variable_y.param.watch(update_variables_row, 'value')
variable_z.param.watch(update_variables_row, 'value')
bubble_scaling.param.watch(update_variables_row, 'value')

variables_row.servable()

              A   B   C  D   E    F    G    H   I    J  ...   P   Q  R    S  \
Date                                                    ...                   
2019-01-01  526   3   9  0  66  472  260  116  45  202  ...  85  38  0   97   
2019-01-02  504  11  10  0  57  472  157  108   3  223  ...  73   0  0   92   
2019-01-03  499   0  10  0  42  475  154  104   0  199  ...  65  20  5   84   
2019-01-04  504   0   7  0  57  464  149   83  17  198  ...  83  19  0   89   
2019-01-05  499  15   9  0  31  466  159   84  14  219  ...  97  31  0  104   

             T  U   V    W   X   Y  
Date                                
2019-01-01  19  7   0  174  55  12  
2019-01-02  45  0  14  132  65   8  
2019-01-03  89  0   4  132  39   6  
2019-01-04   0  0  14  143  57  20  
2019-01-05   0  8  28   46  44  15  

[5 rows x 25 columns]
   Price  Profit   Sales  Marketing  Advertising  Cost
A     21       6  175173      21000     0.119881    15
B     28      16    4489       3000     0.668300    12

BokehModel(combine_events=True, render_bundle={'docs_json': {'3c3d9256-d0b2-48d7-bd9b-0e9306794545': {'version…